In [1]:
#pharmacovigilance signal detection, openfda, me-llama-70b, time series 
#jack baxter, 10/2025

In [ ]:
#load packs
from fastapi import FastAPI, HTTPException
from fastapi.responses import HTMLResponse
import pandas as pd
import plotly.express as px
import plotly.io as pio
import requests
from datetime import datetime
import uvicorn

In [3]:
#create app
app = FastAPI(title='Pharmacovigilance Signal Detection API',version='2.3',
              description='API for detecting pharmacovigilance signals using OpenFDA data and ME-LLaMA model.')

In [ ]:
#initialize input for drug investigation
drugchoice = input('Please enter the name of the drug you want to investigate: ')

In [5]:
#openFDA data pull function 
def get_fda_data(drugchoice):
    url = 'https://api.fda.gov/drug/event.json'
    #between jan 2024 and sept 2025
    query = f'patient.drug.medicinalproduct:"{drugchoice}"+AND+receivedate:[20140101+TO+20250930]'
    params = {'search': query, 'count': 'receivedate', 'limit': 1000}
    #get response with url and params
    resp = requests.get(url, params=params)
    data = resp.json().get('results', [])
    #create df
    df = pd.DataFrame(data)
    if df.empty:
        raise ValueError(f'No data found for drug: {drugchoice}')
    #date in correct format for TS
    df['date'] = pd.to_datetime(df['time'], format='%Y%m%d')
    df['quarter'] = df['date'].dt.to_period('Q').dt.to.timestamp()
    #return date
    return df.sort_values('date')

In [6]:
@app.get("/", response_class=HTMLResponse)
async def home():
    return """
    <h1>🛡️ AEGIS: Real-Time Pharmacovigilance</h1>
    <p>Live FAERS signal detection • Q3 2025 data • <a href="/signal/semaglutide">see Ozempic example</a></p>
    """